**<font size='5'>以下顺次点击下方每个单元格左侧的“运行”图标，不可跳过步骤</font>**
**</br>【重要】:** 务必在"修改"->"笔记本设置"->"硬件加速器"中选择GPU！否则处理速度会非常慢。
 **</br>【IMPORTANT】:** Make sure you select GPU as hardware accelerator in notebook settings, otherwise the processing speed will be very slow.

In [ ]:
#@title **通用参数/Required settings:**


# @markdown **【IMPORTANT】:**<font size="2">Select uploaded file type.
# @markdown **</br>【重要】:** 选择上传的文件类型(视频-video/音频-audio）</font>

# encoding:utf-8
file_type = "audio"  # @param ["audio","video"]

# @markdown #### **Youtube video or playlist**
url = "https://www.bilibili.com/video/BV1HT411K74R"  # @param {type:"string"}
filename="1"  # @param {type:"string"}

model_size = "large-v2"  # @param ["base","small","medium", "large-v1","large-v2"]
language = "en"  # @param ["en","cn"]




In [ ]:
#@title **运行Whisper/Run Whisper**
#@markdown 完成后srt文件将自动下载到本地/srt file will be auto downloaded after finish.
! pip install faster-whisper
! pip install ffmpeg
! pip install torch
! pip install pysubs2
! pip install you-get
from IPython.display import clear_output 
clear_output()
print('语音识别库配置完毕，将开始转换')
import os
import subprocess
import torch
from google.colab import files
from faster_whisper import WhisperModel
from tqdm import tqdm
import time
import pandas as pd
import requests
from urllib.parse import quote_plus
from pathlib import Path
import sys
import pysubs2
import gc
import re

# assert file_name != ""
# assert language != ""
file_basenames = []

torch.cuda.empty_cache()
print('加载模型 Loading model...')
model = WhisperModel(model_size)

语音识别库配置完毕，将开始转换
加载模型 Loading model...


In [ ]:
import sys
from you_get import common as you_get       #导入you-get库
 
#设置下载目录
directory = %pwd  

#设置要下载的视频地址    
sys.argv = ['you-get', '-l', '-o', directory,'-O',f'{filename}', url]
 
you_get.main()


site:                Bilibili
title:               【直播回放】卡尔·弗里斯顿：我和我的马尔可夫毯：能动推理和最小自由能原理
stream:
    - format:        dash-flv480
      container:     mp4
      quality:       清晰 480P
      size:          328.1 MiB (344054083 bytes)
    # download-with: you-get --format=dash-flv480 [URL]

 100% (328.1/328.1MB) ├████████████████████████████████████████┤[2/2]   33 MB/s
Merging video parts... Merged into 1.mp4




In [ ]:

tic = time.time()
print('识别中 Transcribe in progress...')
segments, info = model.transcribe(audio = f'{filename}.mp4',beam_size=5,language=language)

# segments is a generator so the transcription only starts when you iterate over it
# to use pysubs2, the argument must be a segment list-of-dicts
total_duration = round(info.duration, 2)  # Same precision as the Whisper timestamps.
results= []
with tqdm(total=total_duration, unit=" seconds") as pbar:
    for s in segments:
        segment_dict = {'start':s.start,'end':s.end,'text':s.text}
        results.append(segment_dict)
        segment_duration = s.end - s.start
        pbar.update(segment_duration)


#Time comsumed
toc = time.time()
print('识别完毕 Done')
print(f'Time consumpution {toc-tic}s')

srt_file_path = filename+'.srt'
subs = pysubs2.load_from_whisper(results)
subs.save(srt_file_path)

files.download(srt_file_path)

torch.cuda.empty_cache()
print('字幕生成完毕 All done!')

识别中 Transcribe in progress...


 96%|█████████▌| 8088.9000000000115/8437.16 [11:08<00:28, 12.09 seconds/s]

识别完毕 Done
Time consumpution 705.3728559017181s


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

字幕生成完毕 All done!
